# Assess

## `df_archive`

#### Visual assessment
I visually assess `df_archive` by opening the csv file on Google Sheet (it is easier to have a scroll through the datset than by Pandas. First thing I noticed there are a lot of empty field at the columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`. These fields indicate the tweet is a retweet or not. I probably need to drop rows with non-empty field under these columns for the purpose of this analysis. 

Next for the last four columns (`doggo`, `floofer`, `pupper` and  `puppo`), they all represent various stages of dogs so it should be grouped under a single column (untidy data).

Another data quality issue was found under `name` column. There are many invalid names called 'a'.

#### Programmatic assessment

1. Missing data: Missing data for columns related to retweet. (.info())
2. Data type: Erroneous datatypes (`timestamp`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
3. Duplicate data: no duplicate tweet. sum(df_archive.tweet_id.duplicated())
4. Abnormal outliers: Some exceptionally high rating in `rating_numerator`.(.describe()) Mean ratings is 13.1 but max rating is 1776 and some well over serval hundreds. Those are valid ratings, typical for [special occasions](https://twitter.com/dog_rates/status/749981277374128128/photo/1) or images [containing more than one dog](https://twitter.com/dog_rates/status/731156023742988288).

## `df_image`

#### Visual assessment

By visual assessing df_image, I notice there are multiple predictions for each dog. For simplicity, I will only take the most confident dog prediction(`p1` and `p1_dog`=`True`) and won't bother with the rest.

#### Programmatic assessment

1. Missing data: no missing data using .info()
2. Data type: Erroneous datatype for `tweet_id` (.info())
3. Duplicate data: no duplicate tweet (sum(df_image.tweet_id.duplicated())=0)
4. Data Validity: All numeric data are in their valid range (.describe())

### `df_tweet`

#### Visual assessment

Dataset looks good.

#### Programmatic assessment

1. Missing data: no missing data using .info()
2. Data type: Erroneous datatype for `id` (.info())
3. Duplicate data: no duplicate tweet (sum(df_image.tweet_id.duplicated())=0)
4. Data Validity: All numeric data are in their valid range (.describe())

### Quality
##### `df_archive`
- For `name` column, some are of the string "none" and some are named "a"
- "none" instead of empty under `doggo`, `floofer`, `pupper` and  `puppo`
- Erroneous datatypes (`tweet_id`, `timestamp`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
- 181 tweets are retweet (`non_empty retweeted_status_id`)
- Data beyound August 1st, 2017 has no corresponding image prediction

##### `df_image` 
- Irrelevant data except `p1` and `p1_dog == True`
- Erroneous datatype `tweet_id`

#### `df_tweet`
- Erroneous datatype `id`

### Tidiness
- `df_image` should be part of `df_archive` 
- `df_tweet` should be part of `df_archive`

##### `df_archive` 
- One variable in four columns (`doggo`, `floofer`, `pupper` and  `puppo`).
